In [1]:
import pandas as pd
pd.set_option('display.max_columns', 45)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 50)

In [5]:
hist_genes = pd.read_csv('histone_genes.csv',sep=',')
data = pd.read_csv('rawdata.txt', sep='\t')

In [6]:
# extract from 'Alias(es) interactor A' and 'Alias(es) interactor B' between uniprotkb:GENE(gene name)
data['Alias(es) interactor A']=data['Alias(es) interactor A'].replace({r'.*(\|uniprotkb:)(.{0,12})(\(gene name\)\|).*' : r'\2'}, regex=True)
data['Alias(es) interactor B']=data['Alias(es) interactor B'].replace({r'.*(\|uniprotkb:)(.{0,12})(\(gene name\)\|).*' : r'\2'}, regex=True)
data=data.rename({'Alias(es) interactor A' : 'protein1', 'Alias(es) interactor B' : 'protein2'}, axis=1)

In [26]:
# Delete rows with not HGNC identifiers
data=data.loc[~(data['protein1']=='psi-mi:a0a024ra76_human(display_long)|uniprotkb:hCG_1744368(orf name)')&
           ~(data['protein1']=='psi-mi:386682(display_short)|psi-mi:EBI-10171679(display_long)')&
           ~(data['protein1']=='psi-mi:brd4-nut(display_short)|psi-mi:EBI-8101903(display_long)')&
           ~(data['protein1']=='psi-mi:ubiq_human(display_short)|psi-mi:EBI-4399559(display_long)')&
           ~(data['protein2']=='psi-mi:ubiq_human(display_short)|psi-mi:EBI-4399559(display_long)')&
           ~(data['protein2']=='psi-mi:386682(display_short)|psi-mi:EBI-10171679(display_long)')&
           ~(data['protein2']=='psi-mi:sept10(display_short)|psi-mi:EBI-18171702(display_long)')&
           ~(data['protein2']=='psi-mi:yb035_human(display_long)')&
           ~(data['protein2']=='psi-mi:ubiq_human(display_short)|psi-mi:EBI-4399559(display_long)') ]

In [27]:
# Human genes only
data=data.loc[(data['Taxid interactor A']=='taxid:9606(human)|taxid:9606(Homo sapiens)')&(data['Taxid interactor B']=='taxid:9606(human)|taxid:9606(Homo sapiens)')]

In [28]:
# Protein interactions
data=data.loc[(data['Type(s) interactor A']=='psi-mi:"MI:0326"(protein)')&(data['Type(s) interactor B']=='psi-mi:"MI:0326"(protein)')]

In [29]:
# Select only needed columns
data['Interaction detection method(s)']=data['Interaction detection method(s)'].replace({r'(.*(?<="\()(.*?)(?=\)).*)' : r'\2'}, regex=True)
data['Interaction type(s)']=data['Interaction type(s)'].replace({r'(.*(?<="\()(.*?)(?=\)).*)' : r'\2'}, regex=True)
data['Source database(s)']=data['Source database(s)'].replace({r'(.*(?<="\()(.*?)(?=\)).*)' : r'\2'}, regex=True)
data['Confidence value(s)']=data['Confidence value(s)'].replace({r'(.*\:)(.*?)' : r'\2'}, regex=True)
data['Experimental role(s) interactor A']=data['Experimental role(s) interactor A'].replace({r'(.*(?<="\()(.*?)(?=\)).*)' : r'\2'}, regex=True)
data['Experimental role(s) interactor B']=data['Experimental role(s) interactor B'].replace({r'(.*(?<="\()(.*?)(?=\)).*)' : r'\2'}, regex=True)
data=data[['protein1',
         'protein2',
         'Interaction detection method(s)',
         'Interaction type(s)',
         'Source database(s)',
         'Confidence value(s)',
         'Experimental role(s) interactor A',
         'Experimental role(s) interactor B']]

In [32]:
data=pd.merge(data, hist_genes[['Histone type', 'HGNC Symbol', 'Canonicity']], how='left', left_on='protein1', right_on='HGNC Symbol', )
data=data.drop(columns=['HGNC Symbol'])

In [33]:
data_2=data.loc[~(data['protein2'].isin(hist_genes['HGNC Symbol']))]

In [35]:
intact_interactors=data['protein2'].loc[data['Histone type']!='H1'].drop_duplicates()
intact_interactors.to_csv('intact_interactors.csv', index=False, header=True)

intact_interactors_2=data_2['protein2'].loc[data_2['Histone type']!='H1'].drop_duplicates()
intact_interactors_2.to_csv('intact_interactors_2.csv', index=False, header=True)

In [36]:
intact_interaction=data_2.drop_duplicates()
intact_interaction=intact_interaction[['protein1', 'protein2', 'Interaction detection method(s)', 'Confidence value(s)','Source database(s)']]
intact_interaction.to_csv('intact_interaction.csv', index=False, header=True)

In [37]:
# Here processing ends (cell 31). Next 107th cell is analysis